In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os

In [3]:
path = "../output/news/"
df_each = []
for filename in os.listdir(path):
    f = os.path.join(path, filename)
    if os.path.isfile(f):
        temp = pd.read_csv(f)
        temp = temp[temp.name.apply(lambda x: x != "name")]
        df_each.append((temp, filename))

In [4]:
def score(reduc_time, accuracy, train_time, red_w = 200, ac_w = 1, train_w = 20):
    return (accuracy ** ac_w)*(red_w/int(reduc_time + 1) + train_w/int(train_time + 1))

In [5]:
red_w = 10
ac_w = 10
train_w = 10
included_cols = ["name", "filename", "original_shape", "transformed_shape", "params", "reduction_time", "accuracy", "train_time", "score_series"]

In [17]:
out_df = pd.DataFrame(columns=included_cols)
collected_all_df = pd.DataFrame(columns=included_cols)
for res in df_each:
    res_copy = res[0].copy()
    res_copy["filename"] = res[1]
    score_series = res_copy[res_copy.name != "Nothing"][["reduction_time", "train_time", "accuracy"]].apply(lambda x: score(x["reduction_time"], x["accuracy"], red_w, ac_w, train_w), axis=1)
    res_copy_score = res_copy.copy()
    res_copy_score["score_series"] = score_series
    out_df = pd.concat([out_df, res_copy_score])
    max_ind = res_copy_score.groupby(by=["original_shape"])["score_series"].idxmax()
    collected = res_copy_score.iloc[max_ind][["name", "filename", "original_shape", "transformed_shape", "params", "reduction_time", "accuracy", "train_time", "score_series"]]
    collected_all_df = pd.concat([collected_all_df, collected])

In [21]:
all = out_df.groupby(by="filename", group_keys=True).apply(lambda x: x[:])
all[["name", "original_shape", "transformed_shape", "params", "reduction_time", "accuracy", "train_time", "score_series"]]

name  original_shape transformed_shape  \
filename                                                                       
10       0   extremely sparse JL transform  (40886, 62108)     (41896, 5359)   
         1   extremely sparse JL transform  (40886, 62108)     (41896, 5206)   
         2   extremely sparse JL transform  (40886, 62108)     (41896, 2679)   
         3   extremely sparse JL transform  (40886, 62108)     (41896, 2603)   
         4   extremely sparse JL transform  (40886, 62108)      (41896, 535)   
...                                    ...             ...               ...   
5        21                   JL transform  (23183, 40554)      (23688, 221)   
         22                   JL transform  (23183, 40554)       (23688, 88)   
         23                   JL transform  (23183, 40554)       (23688, 68)   
         24                            PCA  (23183, 40554)      (23688, 236)   
         25                        Nothing  (23183, 40554)    (23688, 40554)   

                                                  params  reduction_time  \
filename                                                                   
10       0                      {'ep': 0.05, 'de': 0.05}       22.951098   
         1                       {'ep': 0.05, 'de': 0.1}       23.310290   
         2                       {'ep': 0.1, 'de': 0.05}       22.531250   
         3                        {'ep': 0.1, 'de': 0.1}       22.674591   
         4                       {'ep': 0.5, 'de': 0.05}       22.418684   
...                                                  ...             ...   
5        21                       {'ep': 0.5, 'de': 0.1}        3.134760   
         22                      {'ep': 0.9, 'de': 0.05}        2.704862   
         23                       {'ep': 0.9, 'de': 0.1}        2.657858   
         24  {'n_components': 236, 'svd_solver': 'auto'}       62.303225   
         25                                           {}        0.000032   

             accuracy  train_time  score_series  
filename                                         
10       0      0.408    2.352891      0.000288  
         1      0.613    1.874792      0.016744  
         2      0.669    1.138318      0.040459  
         3      0.471    1.228818      0.001210  
         4      0.947    0.686779      1.306935  
...               ...         ...           ...  
5        21     0.434    0.579391      0.001024  
         22     0.286    0.537779      0.000019  
         23     0.268    0.550330      0.000010  
         24     0.716    0.621871      0.070003  
         25     0.610    7.463229           NaN  

[104 rows x 8 columns]

In [18]:
collected_all_df.groupby(["filename", "name"]).count().sort_values(by="original_shape").groupby(level=0).tail(1).sort_values(by="filename")


,,original_shape,transformed_shape,params,reduction_time,accuracy,train_time,score_series
filename,name,,,,,,,
10,extremely sparse JL transform,1,1,1,1,1,1,1
2,extremely sparse JL transform,1,1,1,1,1,1,1
3,extremely sparse JL transform,1,1,1,1,1,1,1
5,extremely sparse JL transform,1,1,1,1,1,1,1
